# Load data

In [1]:
import pandas as pd

data_fpath_2017 = '../av_survey_data/2017_AV_Public-edited.xls'

data = {}

In [2]:
# Load 2017 data
data[2017] = pd.read_excel(data_fpath_2017, skiprows=[1])
print(len(data[2017]))

# Clean 2017 data: remove 2nd-row headers
float_colnames = ["On a typical day, how safe do you feel using Pittsburgh's streets? (1 being &quot;very unsafe&quot; and 5 being &quot;very safe&quot;)",
                  "Unnamed: 9"]
for colname in float_colnames:
    data[2017][colname] = data[2017][colname].replace('No experience', '0')
    data[2017][colname] = data[2017][colname].astype(float)
data[2017].rename(columns={
    "On a typical day, how safe do you feel using Pittsburgh's streets? (1 being &quot;very unsafe&quot; and 5 being &quot;very safe&quot;)":
        "On a typical day, how safe do you feel using Pittsburgh's streets? with human-driven cars",
    "Unnamed: 9": 
        "On a typical day, how safe do you feel using Pittsburgh's streets? with autonomous vehicles",
    "On public streets, do you think that a regulatory authority should...":
        "On public streets, do you think that a regulatory authority should come up with regulations regarding how AVs are tested?",
    "Unnamed: 12":
        "On public streets, do you think that a regulatory authority should cap the speed limit in which AVs are allowed to operate?",
    "Unnamed: 13":
        "On public streets, do you think that a regulatory authority should prevent AVs from operating in an active school zone?",
    "Unnamed: 14":
        "On public streets, do you think that a regulatory authority should Require companies to share non-personal data with the proper authorities, planning agencies, etc? (AVs collect lots of useful data for city planners)",
}, inplace=True)

data[2017].columns = [col.replace('<br />', '').strip() for col in data[2017].columns]
data[2017].drop(['Status', 'Excluded'], axis=1, inplace=True)

print(data[2017].columns)

813
Index(['Start Date', 'End Date',
       'Have you interacted with an AV while riding your bicycle on the streets of Pittsburgh?',
       'Have you interacted with an AV while using sidewalks and crosswalks in Pittsburgh?',
       'If so, what were the circumstances? What were your observations?',
       'How do you feel right now about the use of Pittsburgh's public streets as a proving ground for AVs?',
       'On a typical day, how safe do you feel using Pittsburgh's streets? with human-driven cars',
       'On a typical day, how safe do you feel using Pittsburgh's streets? with autonomous vehicles',
       'Do you think that AVs have the potential to reduce injuries and fatalities?',
       'On public streets, do you think that a regulatory authority should come up with regulations regarding how AVs are tested?',
       'On public streets, do you think that a regulatory authority should cap the speed limit in which AVs are allowed to operate?',
       'On public streets, do you 

In [3]:
# Load, clean 2019 data

data_fpath_2019 = '../av_survey_data/2019-BikePGH-AV-RAW-TO-SHARE.xlsx'
data[2019] = pd.read_excel(data_fpath_2019, skiprows=[1])
print(len(data[2019]))
data[2019].columns = [col.replace('\xa0', ' ').strip() for col in data[2019].columns]
print(data[2019].columns)

795
Index(['Start Date', 'End Date',
       'To what extent have you been paying attention to the subject of AVs in the news?',
       'How familiar are you with the technology behind autonomous vehicles?',
       'Have you shared the road with an Autonomous Vehicle (AV) while riding your bicycle on the streets of Pittsburgh?',
       'Have you been near an AV while walking or using a mobility device (wheelchair, etc) in Pittsburgh?',
       'If you answered YES to either question, were there notable circumstances? What were your observations?',
       'Please share any positive experiences that you have had with an AV.',
       'Please share any negative experiences that you have had with an AV.',
       'On a typical day, how safe do you feel sharing the road with autonomous vehicles? (1 being very unsafe and 5 being very safe)',
       'On a typical day, how safe do you feel sharing the road with human-driven cars? (1 being very unsafe and 5 being very safe)',
       'What effect do

## Merge 2017 and 2019 data

In [4]:
print(len(data[2017].columns))
print(len(data[2019].columns))

21
27


In [5]:
common_cols = set(data[2017].columns).intersection(set(data[2019].columns))
common_cols

{'End Date',
 'How familiar are you with the technology behind autonomous vehicles?',
 'Start Date',
 'To what extent have you been paying attention to the subject of AVs in the news?'}

In [5]:
common_columns = {
    'start_date': {2017: 'Start Date', 2019: 'Start Date'},
    'end_date': {2017: 'End Date', 2019: 'End Date'},
    'interacted_av_biking': {2017: 'Have you interacted with an AV while riding your bicycle on the streets of Pittsburgh?', 
                             2019: 'Have you shared the road with an Autonomous Vehicle (AV) while riding your bicycle on the streets of Pittsburgh?'},
    'interacted_av_walking': {2017: 'Have you interacted with an AV while using sidewalks and crosswalks in Pittsburgh?', 
                             2019: 'Have you been near an AV while walking or using a mobility device (wheelchair, etc) in Pittsburgh?'},
    'interaction_details': {2017: 'If so, what were the circumstances? What were your observations?',
                            2019: 'If you answered YES to either question, were there notable circumstances? What were your observations?' },
    'positive_av_interaction': {2019: "Please share any positive experiences that you have had with an AV."},
    'negative_av_interaction': {2019: "Please share any negative experiences that you have had with an AV."},
    'thoughts_pgh_av_testing': {2017: "How do you feel right now about the use of Pittsburgh's public streets as a proving ground for AVs?",
                           2019: "What do you think about the use of Pittsburgh's public streets as a proving ground for Autonomous Vehicles?"},
    'feel_safe_cars': {2017: "On a typical day, how safe do you feel using Pittsburgh's streets? with human-driven cars",
                           2019: "On a typical day, how safe do you feel sharing the road with human-driven cars? (1 being very unsafe and 5 being very safe)"},
    'feel_safe_avs': {2017: "On a typical day, how safe do you feel using Pittsburgh's streets? with autonomous vehicles",
                           2019: "On a typical day, how safe do you feel sharing the road with autonomous vehicles? (1 being very unsafe and 5 being very safe)"},
    'av_reduce_injuries': {2017: "Do you think that AVs have the potential to reduce injuries and fatalities?",
                           2019: "What effect do you think that AVs will have on traffic injuries and fatalities?"},
    'regulate_av_testing': {2017: "On public streets, do you think that a regulatory authority should come up with regulations regarding how AVs are tested?"},
    'av_speed_limit': {2017: "On public streets, do you think that a regulatory authority should cap the speed limit in which AVs are allowed to operate?",
                          2019: "On City of Pittsburgh public streets, should AV speeds be capped at 25mph when operating in “autonomous mode”?"},
    'av_school_zone': {2017: "On public streets, do you think that a regulatory authority should prevent AVs from operating in an active school zone?",
                          2019: "On City of Pittsburgh public streets, do you think that AVs should operate in “manual mode” while in an active school zone?"},
    'av_share_data': {2017: "On public streets, do you think that a regulatory authority should Require companies to share non-personal data with the proper authorities, planning agencies, etc? (AVs collect lots of useful data for city planners)",
                          2019: "On City of Pittsburgh public streets, should AV companies be required to share some non-personal data (e.g. number of trips, pickup/drop off locations, number of miles driven) with the proper authorities (e.g. Department of Mobility, PennDOT, Public Safety)?"},
    'other_av_regulations': {2017: "Any other suggested regulations?",
                          2019: "Any other suggested laws or regulations?"},
    'av_two_drivers': {2019: "On City of Pittsburgh public streets, should AVs have two full-time employees (pilot and co-pilot) at all times?"},
    'av_disclose_performance': {2019: "On City of Pittsburgh public streets, should AV companies be required to disclose information and data as to the limitations, capabilities, and real-world performance of their cars with the proper authorities?"},
    'av_report_safety': {2019: "On City of Pittsburgh public streets, should AV companies be required to report all safety-related incidents with the proper authorities, even if a police report isn’t required?"},
    'paying_attention_av_news': {2017: 'To what extent have you been paying attention to the subject of AVs in the news?',
                                2019: 'To what extent have you been paying attention to the subject of AVs in the news?'},
    'familiar_av_tech': {2017: 'How familiar are you with the technology behind autonomous vehicles?',
                        2019: 'How familiar are you with the technology behind autonomous vehicles?'},
    'other_comments': {2017: "Anything else you'd like to share regarding AVs?",
                        2019: "Anything else you'd like to share regarding Autonomous Vehicles?"},
    'herzberg_death_change_opinion': {2019: "In March of 2018, an AV struck and killed Elaine Herzberg, a pedestrian, in Tempe, AZ. As a pedestrian and/or bicyclist, how did this event and its outcome change your opinion about sharing the road with AVs?"},
    'bikepgh_should_advocate': {2017: "Is this an advocacy issue that BikePGH should dedicate resources to?",},
    'bikepgh_position': {2017: "What do you think BikePGH's position on AVs should be?",},
    'elaborate_bikepgh_position': {2017: "Please elaborate on this.",},
    'zipcode': {2017: "Zipcode",
                2019: "Zip Code"},
    'bikegph_member': {2019: "Are you currently an active member of BikePGH?"},
    'own_car': {2019: "Do you (or someone in your household) own an automobile?"},
    'own_smartphone': {2019: "Do you own a smartphone?"},
    'age': {2019: "What is your age?"},
                     }

len(common_columns)

# Make a dataframe
pd.set_option("display.max_colwidth", -1)
colnames = pd.DataFrame(common_columns).transpose()
colnames

,2017,2019
age,NaN,What is your age?
av_disclose_performance,NaN,"On City of Pittsburgh public streets, should AV companies be required to disclose information and data as to the limitations, capabilities, and real-world performance of their cars with the proper authorities?"
av_reduce_injuries,Do you think that AVs have the potential to reduce injuries and fatalities?,What effect do you think that AVs will have on traffic injuries and fatalities?
av_report_safety,NaN,"On City of Pittsburgh public streets, should AV companies be required to report all safety-related incidents with the proper authorities, even if a police report isn’t required?"
av_school_zone,"On public streets, do you think that a regulatory authority should prevent AVs from operating in an active school zone?","On City of Pittsburgh public streets, do you think that AVs should operate in “manual mode” while in an active school zone?"
av_share_data,"On public streets, do you think that a regulatory authority should Require companies to share non-personal data with the proper authorities, planning agencies, etc? (AVs collect lots of useful data for city planners)","On City of Pittsburgh public streets, should AV companies be required to share some non-personal data (e.g. number of trips, pickup/drop off locations, number of miles driven) with the proper authorities (e.g. Department of Mobility, PennDOT, Public Safety)?"
av_speed_limit,"On public streets, do you think that a regulatory authority should cap the speed limit in which AVs are allowed to operate?","On City of Pittsburgh public streets, should AV speeds be capped at 25mph when operating in “autonomous mode”?"
av_two_drivers,NaN,"On City of Pittsburgh public streets, should AVs have two full-time employees (pilot and co-pilot) at all times?"
bikegph_member,NaN,Are you currently an active member of BikePGH?
bikepgh_position,What do you think BikePGH's position on AVs should be?,NaN


In [5]:
# Save out colnames
out_fpath = '../av_survey_data/column_mappings.csv'
colnames.to_csv(out_fpath, index_label='transformed_column_name')

In [24]:
data[2019].columns

Index(['Start Date', 'End Date',
       'To what extent have you been paying attention to the subject of AVs in the news?',
       'How familiar are you with the technology behind autonomous vehicles?',
       'Have you shared the road with an Autonomous Vehicle (AV) while riding your bicycle on the streets of Pittsburgh?',
       'Have you been near an AV while walking or using a mobility device (wheelchair, etc) in Pittsburgh?',
       'If you answered YES to either question, were there notable circumstances? What were your observations?',
       'Please share any positive experiences that you have had with an AV.',
       'Please share any negative experiences that you have had with an AV. ',
       'On a typical day, how safe do you feel sharing the road with autonomous vehicles? (1 being very unsafe and 5 being very safe)',
       'On a typical day, how safe do you feel sharing the road with human-driven cars? (1 being very unsafe and 5 being very safe)',
       'What effect do yo

In [1]:
# Check column data types

for col in common_columns:
    for key, colname in common_columns[col].items():
        print(f'{key} {colname}')
        print(data[key][colname].dtype)
#         print(data[key][colname].head())
        print()
        
    print()

NameError: name 'common_columns' is not defined

In [6]:
# Replace column names

replacements_2019 = {val[2019]: key for key, val in common_columns.items() if 2019 in val.keys()}
replacements_2019

{'Any other suggested laws or regulations?': 'other_av_regulations',
 "Anything else you'd like to share regarding Autonomous Vehicles?": 'other_comments',
 'Are you currently an active member of BikePGH?': 'bikegph_member',
 'Do you (or someone in your household) own an automobile?': 'own_car',
 'Do you own a smartphone?': 'own_smartphone',
 'End Date': 'end_date',
 'Have you been near an AV while walking or using a mobility device (wheelchair, etc) in Pittsburgh?': 'interacted_av_walking',
 'Have you shared the road with an Autonomous Vehicle (AV) while riding your bicycle on the streets of Pittsburgh?': 'interacted_av_biking',
 'How familiar are you with the technology behind autonomous vehicles?': 'familiar_av_tech',
 'If you answered YES to either question, were there notable circumstances? What were your observations?': 'interaction_details',
 'In March of 2018, an AV struck and killed Elaine Herzberg, a pedestrian, in Tempe, AZ. As a pedestrian and/or bicyclist, how did this eve

In [7]:
data[2019].columns = [replacements_2019[colname] for colname in data[2019].columns]
data[2019].head()

,start_date,end_date,paying_attention_av_news,familiar_av_tech,interacted_av_biking,interacted_av_walking,interaction_details,positive_av_interaction,negative_av_interaction,feel_safe_avs,...,av_disclose_performance,av_report_safety,other_av_regulations,herzberg_death_change_opinion,other_comments,zipcode,bikegph_member,own_car,own_smartphone,age
0,2019-02-02 08:08:47,2019-02-02 08:12:18,To a moderate extent,Somewhat familiar,Yes,Yes,The AV stopped with plenty of room,NaN,NaN,4.0,...,Yes,Yes,NaN,No change,"I think it’s terrible that Elaine was hit, but after reading a lot about it, I firmly believe she would have been hit had it been a human operating the car and not an autonomous vehicle. I see countless people on their phones driving daily and I feel much more comfortable with AVs than distracted drivers.",15212,No,Yes,Yes,25-34
1,2019-02-02 01:57:34,2019-02-02 02:03:24,To a moderate extent,Somewhat familiar,Yes,No,When approaching from the rear the vehicle matched my speed and kept a constsnt follow distance this is unnerving and I would always rather have a car pass me. I interpreted this behavior as 'stalking' or 'following' until turning around and realizing the vehicle was autonomous,They usually detect my presence better than human drivers.,Weird and sometimes disruptive traffic behaviors,5.0,...,Yes,Not sure,NaN,No change,Every single AV is already a better driver then even the best human driver. They dont need to be perfect they just need to be better and they already are.,15232,Not sure,Yes,Yes,25-34
2,2019-02-01 19:06:59,2019-02-01 19:10:54,To a moderate extent,Somewhat familiar,Yes,Yes,NaN,NaN,NaN,1.0,...,No,Not sure,NaN,Significantly more negative opinion,NaN,NaN,No,No,No,NaN
3,2019-02-01 14:28:39,2019-02-01 14:40:48,To a moderate extent,Extremely familiar,Yes,Yes,"I was in a crosswalk, it stopped for me but it was an awkward situation because it stopped after it had continued into the intersection. It made me feel very uncomfortable.",NaN,I've experienced them while on my bike when I lived in San Francisco. I encountered one almost every day and didn't like it. Nothing negative happened when I did encounter them but I don't feel comfortable around them. I don't trust them and I don't trust the possibility of surveillance by them when I'm nearby.,2.0,...,Yes,Yes,Ban them outright,No change,NaN,15136,No,No,Yes,55-64
4,2019-02-01 13:59:09,2019-02-01 14:04:43,To a moderate extent,Mostly familiar,Yes,No,No,None. Not bad experiences either.,NaN,5.0,...,Yes,No,No,No change,"People-driven cars have killed many more than robot-driven cars. And outside of bicycle safety, why is BikePGH taking this up rather than the Department of Mobility and Infrastructure? Oh, that’s because their director hasn’t done jack shit in the couple years she’s been in the city. Oh well.",15201,Yes,No,Yes,35-44


In [11]:
data[2019].shape

(795, 27)

In [2]:
replacements_2017 = {val[2017]: key for key, val in common_columns.items() if 2017 in val.keys()}
data[2017].columns = [replacements_2017[colname] for colname in data[2017].columns]
# data[2017].head()

NameError: name 'common_columns' is not defined

In [9]:
data[2017]['year'] = [2017] * len(data[2017])
data[2019]['year'] = [2019] * len(data[2019])

In [10]:
# Merge 2019 and 2017
data['all'] = pd.concat([data[2017], data[2019]]).reset_index(drop=True)
print(data['all'].shape)
print(data['all'].columns)

(1608, 32)
Index(['age', 'av_disclose_performance', 'av_reduce_injuries',
       'av_report_safety', 'av_school_zone', 'av_share_data', 'av_speed_limit',
       'av_two_drivers', 'bikegph_member', 'bikepgh_position',
       'bikepgh_should_advocate', 'elaborate_bikepgh_position', 'end_date',
       'familiar_av_tech', 'feel_safe_avs', 'feel_safe_cars',
       'herzberg_death_change_opinion', 'interacted_av_biking',
       'interacted_av_walking', 'interaction_details',
       'negative_av_interaction', 'other_av_regulations', 'other_comments',
       'own_car', 'own_smartphone', 'paying_attention_av_news',
       'positive_av_interaction', 'regulate_av_testing', 'start_date',
       'thoughts_pgh_av_testing', 'year', 'zipcode'],
      dtype='object')


In [25]:
# Postprocessing text columns
text_colnames = [
    'interaction_details',
    'positive_av_interaction',
    'negative_av_interaction',
    'other_av_regulations',
    'elaborate_bikepgh_position',
    'other_comments',
                ]

for colname in text_colnames:
    data['all'][colname] = data['all'][colname].str.replace('<br />', ' ')

In [28]:
# Save out
out_fpath = '../av_survey_data/bikepgh_av_survey.csv'
data['all'].to_csv(out_fpath, index_label='participant_id')